In [3]:
import pandas as pd 
import re
from top2vec  import Top2Vec
import spacy
import nltk
import re
import string
import pandas as pd
import numpy as np
import gensim
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import json
%matplotlib inline
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [4]:
newline = []

textstring = ''
with open('Hansard_preCovid.csv', 'r', encoding='utf-16-le') as inp, open('newtextfile.txt', 'w') as out:
    for line in inp:
      newline.append(re.findall(r'"(.*)"', line))
      
temp = pd.Series(newline)

initialDF_preCovid = pd.DataFrame({'Text': temp})
clearWords = stopwords.words('english')
# new_Stopwords = ['in', 'the', 'said', 'like','must', 'many', 'also', '(', ')', '\'', ':', '?', ']', '[', '-', 'mr.', 'mrs.']
lemma= WordNetLemmatizer()
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].astype(str).str.lower()
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].str.replace("|".join([r"^.*?speaker,", r"^.*?chair," ]), ' ', regex=True)
initialDF_preCovid['text_tokens'] = initialDF_preCovid['Text'].apply(word_tokenize)
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x: [item for item in x if item not in clearWords])
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x: [lemma.lemmatize(item) for item in x]) 
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda w: [item for item in w if len(item)>2])
initialDF_preCovid['Text'] = initialDF_preCovid['text_tokens'].apply(lambda w: [' '.join(w) ])

# do we love me or what? isn't this join thing amazing?

In [5]:

# finding bigrams and trigrams mid-covid 

gram_wordlist = ['worker' , 'immigration',
                 'caregiver', 'caregivers', 'canada', 'canadian', 'temporary', 'migrant', 'foreign', 'labour', 'committee', 'family', 'immigrant', 
                 'industry', 'farm','farmer', 'skill', 'skilled', 'bill', 'resident', 'farmer', 'agriculture', 'housing',
                   'employment', 'insurance' ,'employee' , 'harassment', 'residency' ,'residence', 'social', 'development', 'farming', 'program']
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents(initialDF_preCovid['text_tokens'])
finder.apply_freq_filter(15)
bigram_scores = finder.score_ngrams(bigram_measures.pmi)


trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = nltk.collocations.TrigramCollocationFinder.from_documents(initialDF_preCovid['text_tokens'])
finder.apply_freq_filter(15)
trigram_scores = finder.score_ngrams(trigram_measures.pmi)
bigram_pmi = pd.DataFrame(bigram_scores)
bigram_pmi.columns = ['bigram', 'pmi']
bigram_pmi.sort_values(by='pmi', axis = 0, ascending = True, inplace = True)
trigram_pmi = pd.DataFrame(trigram_scores)
trigram_pmi.columns = ['trigram', 'pmi']
trigram_pmi.sort_values(by='pmi', axis = 0, ascending = True, inplace = True)
# Filter for bigrams with only noun-type structures
def bigram_filter(bigram):
    tag = nltk.pos_tag(bigram)
    if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['NN']:
        return False
    if bigram[0] in clearWords or bigram[1] in clearWords:
        return False
    if 'n' in bigram or 't' in bigram:
        return False
    if 'PRON' in bigram:
        return False
    return True
# Filter for trigrams with only noun-type structures
def trigram_filter(trigram):
    tag = nltk.pos_tag(trigram)
    if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['JJ','NN']:
        return False
    if trigram[0] in clearWords or trigram[-1] in clearWords or trigram[1] in clearWords:
        return False
    if 'n' in trigram or 't' in trigram:
         return False
    if 'PRON' in trigram:
        return False
    return True 
# Can set pmi threshold to whatever makes sense - eyeball through and select threshold where n-grams stop making sense
# choose top 500 ngrams in this case ranked by PMI that have noun like structures
filtered_bigram = bigram_pmi[bigram_pmi.apply(lambda bigram:\
                                              bigram_filter(bigram['bigram'])\
                                              and bigram.pmi > 5, axis = 1)][:200]

filtered_trigram = trigram_pmi[trigram_pmi.apply(lambda trigram: \
                                                 trigram_filter(trigram['trigram'])\
                                                 and trigram.pmi >1, axis = 1)][:200]


# bigrams = [' '.join(x) for x in filtered_bigram.bigram.values if x[0] in gram_wordlist or x[1] in gram_wordlist or x[-1] in gram_wordlist ]
bigrams = [' '.join(x) for x in filtered_bigram.bigram.values] 
trigrams = [' '.join(x) for x in filtered_trigram.trigram.values ]

In [7]:

# Concatenate n-grams
def replace_ngram(x): 
    #first convert to string from list, then add the bigram and trigram strings. 
    for gram in trigrams:
        x = x.replace(gram, '_'.join(gram.split(' ')))
    for gram in bigrams:
        x = x.replace(gram, '_'.join(gram.split(' ')))

    print(x)
    return x 


# Filter for only nouns adjectives and verbs
def noun_only(x):
    pos_comment = nltk.pos_tag(x)
    filtered = [word[0] for word in pos_comment if word[1] in ['NN', 'NNS', 'JJ', 'RB', 'RBR', 'RBS']]
    # to filter both noun and verbs
    #filtered = [word[0] for word in pos_comment if word[1] in ['NN','VB', 'VBD', 'VBG','JJS','JJR', 'VBD', 'VBP', 'VBG', 'VBN', 'VBZ' 'VBN', 'VBZ']]
    return filtered

In [ ]:
initialDF_preCovid['Text']= initialDF_preCovid['Text'].map(lambda x: replace_ngram(str(x)))

In [ ]:
# #initialDF_midCovid['Text'] = initialDF_midCovid['Text'].apply(lambda w: [item.translate(table) for item in w])
# initialDF_preCovid['Text'] = initialDF_preCovid['Text'].apply(word_tokenize)
# initialDF_preCovid['Text'] = initialDF_preCovid['Text'].apply(lambda x: [item for item in x if item not in clearWords])
# initialDF_preCovid['Text'] = initialDF_preCovid['Text'].apply(lambda x: [lemma.lemmatize(item) for item in x])
# initialDF_preCovid['Text'] = initialDF_preCovid['Text'].apply(lambda x: [item for item in x if len(item) > 2])

In [10]:
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].apply(word_tokenize)
final_text = initialDF_preCovid.Text.map(lambda x: noun_only(x))

In [30]:
preCovid_t2v_ngrams = Top2Vec(final_text.astype(str).values, embedding_model='doc2vec')


2023-03-27 18:37:39,713 - top2vec - INFO - Pre-processing documents for training
2023-03-27 18:37:40,612 - top2vec - INFO - Creating joint document/word embedding
2023-03-27 18:37:45,583 - top2vec - INFO - Creating lower dimension embedding of documents
2023-03-27 18:38:08,098 - top2vec - INFO - Finding dense areas of documents
2023-03-27 18:38:08,122 - top2vec - INFO - Finding topics


In [31]:
preCovid_t2v_ngrams.topic_words

array([['experience', 'term', 'need', 'area', 'increase', 'day', 'well',
        'skill', 'english', 'long', 'part', 'work', 'new', 'population',
        'week', 'really', 'lot', 'strong', 'help', 'home', 'challenge',
        'hard', 'atlantic_canada', 'able', 'economic', 'economy',
        'forward', 'better', 'newcomer', 'region', 'immigration',
        'great', 'we', 'social', 'something', 'place', 'meet', 'life',
        'sure', 'good', 'people', 'also', 'opportunity', 'way', 'mean',
        'future', 'important', 'employment', 'immigrant', 'care'],
       ['deal', 'significant', 'trade_agreement', 'nafta', 'rule',
        'trade_deal', 'cptpp', 'ndp', 'right', 'trade', 'however',
        'already', 'tariff', 'investor', 'environmental', 'agreement',
        'ceta', 'chapter', 'kind', 'look', 'actually', 'environment',
        'commitment', 'clear', 'labour', 'concern', 'mean', 'export',
        'real', 'tpp', 'party', 'canadian', 'important', 'good',
        'sector', 'provision',

In [26]:
alpha, beta = preCovid_t2v_ngrams.similar_words(keywords=["temporary"], keywords_neg=[], num_words=40)
for word, score in zip(alpha, beta):
    print(f"{word} {score}")

year 0.8892796753430459
official 0.8783397492269205
prime_minister 0.8666422028369025
currently 0.856584572990688
organization 0.8477927169998403
last 0.8446157661274709
contribution 0.8320069044396693
amount 0.8140654562491578
position 0.8134462408157496
many 0.8040621063451541
much 0.8026068207422012
program 0.7994448135016263
result 0.7968932132562376
project 0.7929614567094898
date 0.7924094926494736
full 0.7893474321549279
question 0.7873414660817412
benefit 0.7871700374628603
total 0.787033445037252
iii 0.7832089417669064
department 0.7817440356645966
plan 0.779703790805166
name 0.7795446805815314
minister 0.7793770955455213
quebec 0.7762000601414939
expenditure 0.7737430330235139
average 0.7714615661698017
number 0.7704191246389597
initiative 0.7700405691210462
budget 0.7694348935917195
mr 0.7685395146775065
broken 0.7678978575862714
security 0.7656057167228945
individual 0.7596009915183541
funding 0.7554252019202496
matter 0.7536152806144073
regard 0.7513386554370246
reason 0.7

In [ ]:
midCovid_t2v_ngrams = Top2Vec(final_text.values, embedding_model='doc2vec')
midCovid_t2v_ngrams.topic_words

In [ ]:
topic_words, word_scores, topic_scores, topic_nums = midCovid_t2v_ngrams.search_topics(keywords=["foreign_workers"], num_topics=2)